In [1]:
import wandb
from naip_cnn import augment
import dateutil
import datetime

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
default_augment = augment.Augment(
    augment.RandomFlip(),
    augment.RandomContrast(),
    augment.RandomBrightness(),
)

In [3]:
api = wandb.Api()

In [4]:
def update_run_config(run):
    run_date = dateutil.parser.parse(run.metadata["startedAt"])
    cfg = run.config

    if run_date >= datetime.datetime(2024, 6, 3, 21):
        augmenter = None
    else:
        augmenter = default_augment

    n_train = cfg["data"]["n_samples"]["train"]
    n_val = cfg["data"]["n_samples"]["val"]
    train_path = cfg["data"]["path"]
    val_path = train_path.replace("_train", "_val")

    cfg["data"]["train"] = {
        "n_samples": n_train,
        "augmentation": augmenter.to_json() if augmenter is not None else None,
        "path": train_path,
    }
    cfg["data"]["val"] = {
        "n_samples": n_val,
        "path": val_path,
    }

    cfg["data"]["footprint"]["units"] = "meters"

    del cfg["data"]["n_samples"]
    del cfg["data"]["path"]

In [5]:
runs = api.runs("aazuspan-team/naip-cnn")

for run in runs:
    update_run_config(run)
    run.update()